<img src="logo.png"></img>

## Load data

In [ ]:
import pandas as pd
import re

common_columns = ['counter', 'start_time', 'end_time', 'user', 'vehicle', 'km_start', 'km_end', 'km', 'comments']
v1_columns = common_columns + ['purpose', 'rebound', 'people', 'vehicle_counter', 'duration']
v2_columns = common_columns + ['initial_start_time', 'initial_end_time', 'initial_shift']

counter_converter = lambda x: pd.to_numeric(re.sub(r'^\w+ #', '', x.removeprefix('#REF!')))
km_converter = lambda x: pd.to_numeric(x.removesuffix(' km'))
common_converters = {
    'counter': counter_converter,
    'km_start': km_converter,
    'km_end': km_converter,
    'km': km_converter,
}

v1_trips = pd.read_csv(
    'v1_trips.csv',
    header=3,
    names=v1_columns,
    converters=common_converters,
    parse_dates=['start_time', 'end_time'],
    date_format='%d/%m/%Y %H:%M:%S')
v2_trips = pd.read_csv(
    'v2_trips.csv',
    header=3,
    names=v2_columns,
    converters=common_converters,
    parse_dates=['start_time', 'end_time', 'initial_start_time', 'initial_end_time'],
    date_format='%Y-%m-%d %H:%M:%S')

trips = pd.concat([v1_trips, v2_trips])

## Compute

In [ ]:
def duration(trip):
    if trip['start_time'] != trip['end_time']:
        return trip['end_time'] - trip['start_time']
    else:
        return pd.NaT
trips['duration'] = trips.apply(duration, axis=1)

trips['speed'] = trips['km'] / (trips['duration'] / pd.Timedelta(hours=1))

## Invalid data

In [ ]:
def invalid(trip):
    reasons = []

    if trip['km'] <= 0:
        reasons.append('low km')
    elif trip['km'] >= 1000:
        reasons.append('high km')

    if trip['start_time'] == trip['end_time']:
        reasons.append('identical start and end time')

    if trip['duration'] <= pd.Timedelta(0):
        reasons.append('low duration')
    elif trip['duration'] >= pd.Timedelta(days=7):
        reasons.append('high duration')

    if trip['speed'] <= 3:
        reasons.append('low speed')
    elif trip['speed'] >= 150:
        reasons.append('high speed')

    return ', '.join(reasons)
trips['invalid'] = trips.apply(invalid, axis=1)

invalid_trips = (trips.query('invalid != "" and initial_shift.isnull()')
                      .drop(columns=trips.filter(regex='^initial_')))
invalid_trips.insert(0, 'invalid', invalid_trips.pop('invalid'))
invalid_trips.to_csv('invalid.csv', index=False)

invalid_trips['invalid'].value_counts()

## Statistics

In [ ]:
trips.describe()